In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset,random_split,SubsetRandomSampler
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
from tqdm.auto import tqdm
import random
from torch.utils.tensorboard import SummaryWriter
# from tensorboardX import SummaryWriter

## Dataset

In [3]:
train_data_index = pd.read_csv('../input/classify-leaves/train.csv')
test_data_index = pd.read_csv('../input/classify-leaves/test.csv')

In [4]:
categories = train_data_index['label'].unique()
categories[:10]

array(['maclura_pomifera', 'ulmus_rubra', 'broussonettia_papyrifera',
       'prunus_virginiana', 'acer_rubrum', 'cryptomeria_japonica',
       'staphylea_trifolia', 'asimina_triloba', 'diospyros_virginiana',
       'tilia_cordata'], dtype=object)

In [5]:
train_data_index['label'] = train_data_index['label'].apply(lambda x:int(np.where(categories==x)[0]))

In [6]:
train_data_index.iloc[1]['label']

0

In [7]:
test_tfm = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((224,224)),
    # You may add some transforms here.
    transforms.ColorJitter(brightness=0.5,saturation=0.3),
    transforms.RandomRotation(degrees=(0, 180)),
    transforms.RandomHorizontalFlip(p=0.5),
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
])

In [8]:
class LeavesDataset(Dataset):
    def __init__(self,data_index,tfm,root='../input/classify-leaves/',is_train=True):
        super(LeavesDataset).__init__()
        self.data_index = data_index
        self.root = root
        self.is_train = is_train
        self.tfm = tfm
    def __len__(self):
        return self.data_index.shape[0]
    def __getitem__(self,idx):
        if self.is_train:
            img_name = self.data_index.iloc[idx][0]
            label = self.data_index.iloc[idx][1]
        else:
            img_name = self.data_index.iloc[idx][0]
            label = -1
        img_path = os.path.join(self.root,img_name)
        img = Image.open(img_path)
        img_tranform = self.tfm(img)
        return img_tranform,label
    

In [9]:
# dataset = LeavesDataset(test_data_index,test_tfm,is_train=False)
# import matplotlib.pyplot as plt
# fix, axs = plt.subplots(ncols=2, squeeze=False)
# img = transforms.ToPILImage()(dataset[1000][0])
# axs[0, 0].imshow(np.asarray(img))

## model

In [10]:
# model = models.vgg11_bn()
# model.classifier[6] = nn.Linear(in_features=4096, out_features=categories.shape[0], bias=True)
# # model

In [11]:
def get_model():
    model = models.resnet34(pretrained=True)
    # model.fc = nn.Linear(in_features=4096, out_features=categories.shape[0], bias=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, out_features=176)
    return model

In [12]:
# get_model()

In [13]:
# from torchsummary import summary
# summary(model, input_size=(1,3,224,224))

## mixup

In [14]:
def mixup_data(x, y, device, alpha=1.0):

    '''Compute the mixup data. Return mixed inputs, pairs of targets, and lambda'''
    if alpha > 0.:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1.
    batch_size = x.size()[0]
    
    index = torch.randperm(batch_size).to(device)
    no_mixup_indice = torch.arange(0.4*batch_size).to(index.dtype)    # 保留40%不进行 mix up
    index[:len(no_mixup_indice)] = no_mixup_indice

    mixed_x = lam * x + (1 - lam) * x[index,:]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(y_a, y_b, lam):
    return lambda criterion, pred: lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

## train

In [15]:
lr,epoch_nums,patience = 3e-4,30,50
batch_size = 128

In [16]:
model = get_model()
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [17]:
dataset = LeavesDataset(train_data_index,train_tfm)

indices = np.arange(len(dataset))
np.random.shuffle(indices)

train_sampler = SubsetRandomSampler(indices[len(indices)//5:])
valid_sampler = SubsetRandomSampler(indices[:len(indices)//5])

train_iter = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                       sampler=train_sampler,num_workers=0, pin_memory=True)
valid_iter = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                            sampler=valid_sampler,num_workers=0, pin_memory=True)

In [18]:
best_acc = 0
stale = 0

In [19]:
def try_gpu(i=0): 
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')
device = try_gpu()
print(f'device is {device}')

device is cuda:0


In [20]:
%load_ext tensorboard
# 可视化
writer = SummaryWriter("./runs")

In [21]:
model = model.to(device)

In [22]:
for epoch in range(epoch_nums):
    correct = 0
    total = 0
    train_loss = []
    model.train()
    for x,y in tqdm(train_iter):    
        inputs, targets = x.to(device), y.to(device)
        
        inputs, targets_a, targets_b, lam = mixup_data(inputs, targets, device, alpha=0.5)
        
        optimizer.zero_grad()
        outputs = model(inputs)

        loss_func = mixup_criterion(targets_a, targets_b, lam)
        loss = loss_func(criterion, outputs)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += lam * predicted.eq(targets_a.data).cpu().sum() + (1 - lam) * predicted.eq(targets_b.data).cpu().sum()
        
        train_loss.append(loss.item())
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = correct/total
    
    valid_acc = []
    valid_loss = []
    model.eval()
    for x,y in tqdm(valid_iter):

        with torch.no_grad():
            y_predict = model(x.to(device))
        
        loss = criterion(y_predict,y.to(device))
        acc = (y_predict.argmax(dim=-1) == y.to(device)).float().mean()
        
        valid_loss.append(loss.item())
        valid_acc.append(acc)
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_acc) / len(valid_acc)
    
    writer.add_scalars('loss', {'train_loss':train_loss,
                                'valid_loss':valid_loss}, epoch)
    writer.add_scalars('acc', {'train_acc':train_acc,
                               'valid_acc':valid_acc}, epoch)
    
    if valid_acc>best_acc:
        best_acc = valid_acc
        stale = 0
        # save_model
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_acc': valid_acc,
            }, f"model_best.pth.tar")
        print(f'At epoch {epoch} get better acc model[{valid_acc}]----->model_best.pth.tar')   
    else:
        stale+=1
        if stale > patience:
            print(f'At epoch {epoch} stop early because of {stale} epochs no improve')
            break
            
# save_model
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'best_acc': valid_acc,
    }, f"model_final_best.pth.tar")
writer.close()

  0%|          | 0/115 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

At epoch 0 get better acc model[0.47278469800949097]----->model_best.pth.tar


  0%|          | 0/115 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

...

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/115 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/115 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

In [23]:
%tensorboard --logdir ./runs

## predict

In [24]:
model = get_model()

model = model.to(device)
checkpoint = torch.load(f"model_final_best.pth.tar")
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [25]:
dataset = LeavesDataset(test_data_index,test_tfm,is_train=False)

test_iter = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                       shuffle=False,num_workers=0, pin_memory=True)

In [26]:
label_list = []

In [27]:
model.eval()
for x,y in tqdm(test_iter):
    with torch.no_grad():
        y_predict = model(x.to(device))
    label_list += y_predict.argmax(dim=-1).reshape(-1).tolist()


  0%|          | 0/69 [00:00<?, ?it/s]

In [28]:
test_data_index['label']=categories[label_list]
test_data_index.to_csv("submission.csv",index = False)

In [29]:
!zip -r my.zip ./runs

  adding: runs/ (stored 0%)
  adding: runs/acc_valid_acc/ (stored 0%)
  adding: runs/acc_valid_acc/events.out.tfevents.1660403421.5bf11ec29df8.23.4 (deflated 51%)
  adding: runs/events.out.tfevents.1660403157.5bf11ec29df8.23.0 (deflated 5%)
  adding: runs/loss_valid_loss/ (stored 0%)
  adding: runs/loss_valid_loss/events.out.tfevents.1660403421.5bf11ec29df8.23.2 (deflated 51%)
  adding: runs/acc_train_acc/ (stored 0%)
  adding: runs/acc_train_acc/events.out.tfevents.1660403421.5bf11ec29df8.23.3 (deflated 50%)
  adding: runs/loss_train_loss/ (stored 0%)
  adding: runs/loss_train_loss/events.out.tfevents.1660403421.5bf11ec29df8.23.1 (deflated 52%)
